In [ ]:
# inport BeautifulSoup
import re
from bs4 import BeautifulSoup

In [ ]:
file=open("20170520/a.html", "r+")
soup = BeautifulSoup(file.read(), "html.parser")

In [ ]:
print(soup.find_all("div","playerInfo"))

In [ ]:
for div in soup.find_all("div","playerInfo"):
    # get String
    line = div.get_text()
    
    # parsing String
    line = line.replace(':\n                \n\n ', ':')
    line = line.replace('\n\n\n', '')
    line = line.replace('\n\n                        1', '\n1')
    line = line.replace('\n\n                        2', '\n2')
    line = line.replace('\n\n                        3', '\n3')
    line = line.replace('TSI = ','\nTSI:')
    line = line.replace('\xa0', '')
    line = line.replace(', 임금 = ', '\n임금:')
    line = line.replace('Won', ' Won\n')
    line = line.replace('form, ', 'form\n')
    if line.find('체력 [') > -1 :
        line = line.replace('체력 [', '체력\n특수능력:')
        line = line.replace(']', '\n')
    else :
        line = line.replace('체력', '체력\n특수능력:없음\n')
    line = line.replace('경험과 ', '경험\n')
    line = line.replace('리더십 능력', '리더십능력\n')
    line = line.replace('\n\n', '\n')
    line = line.replace('\n                    ', '')
    line = line.replace('충성도 ', '충성도:')
    line = line.replace('골키핑', '\n골키핑')
    line = line.replace('수비력','\n수비력')
    line = line.replace('플레이메이킹', '\n플레이메이킹')
    line = line.replace('윙어', '\n윙어')
    line = line.replace('패스력','\n패스력')
    line = line.replace('득점력', '\n득점력')
    line = line.replace('세트 피스','\n세트피스')
    
    line = line.replace('신의 경지','20')
    line = line.replace('초월적', '19')
    line = line.replace('마술적', '18')
    line = line.replace('전설적', '17')
    line = line.replace('외계인', '16')
    line = line.replace('막강함', '15')
    line = line.replace('초자연적', '14')
    line = line.replace('세계 최고', '13')
    line = line.replace('훌륭함', '12')
    line = line.replace('눈부심', '11')
    line = line.replace('걸출함', '10')
    line = line.replace('강력함', '9')
    line = line.replace('뛰어남', '8')
    line = line.replace('우수', '7')
    line = line.replace('무난함', '6')
    line = line.replace('불충분함', '5')
    line = line.replace('약함', '4')
    line = line.replace('나쁨', '3')
    line = line.replace('형편 없음', '2')
    line = line.replace('최악', '1')
    line = line.replace('없음', '0')
    
    line = line.replace('(\n', '(')
    line = line.replace('\n (', ':')
    line = line.replace(')', '')
    
    for lines in line.split('\n'):
        if (re.match('[0-9] ', lines)) :
            revertString=lines.split(' ')
            print (revertString[1] + ':' + revertString[0])
        else : 
            print(lines)

In [ ]:
lines = soup.find('div','playerInfo').get_text()

In [ ]:
lines

In [ ]:
lines = lines.replace(':\n                \n\n ', ':')
lines = lines.replace('\n\n\n', '')
lines = lines.replace('\n\n                        1', '\n1')
lines = lines.replace('\n\n                        2', '\n2')
lines = lines.replace('\n\n                        3', '\n3')
lines = lines.replace('TSI','\nTSI')
lines = lines.replace('\xa0', '')
lines = lines.replace(', 임', '\n임')
lines = lines.replace('Won', ' Won\n')
lines = lines.replace('form, ', 'form\n')
lines = lines.replace('체력', '체력\n')
lines = lines.replace('경험과 ', '경험\n')
lines = lines.replace('리더십 능력', '리더십 능력\n')
lines = lines.replace('\n\n', '\n')
lines = lines.replace('\n                    ', '')

In [ ]:
lines.find('체력 [')

In [ ]:
print(lines)